In [155]:
import pandas as pd
import pandasql as ps

## Reding the file and showing few rows to be sure everything is ok ##

In [156]:
# reading the CSV file
csvFile = pd.read_csv('Chicago_crime_mini.csv', delimiter=";")

# displaying the contents of the CSV file
csvFile.head(2)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,False,False,1111,11,28,25,04B,1144606.0,1903566.0,2015,02/10/2018 15:50,41.891.398.861,-87.744.384.567,"(41.891398861, -87.744384567)"
1,10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,True,False,725,7,15,67,26,1166468.0,1860715.0,2015,02/10/2018 15:50,41.773.371.528,-87.665.319.468,"(41.773371528, -87.665319468)"


## Question 1 : what is the year with the highest offence count? ##


In [3]:
q2= """SELECT count(CAST(year as int)) as Count, CAST(year as int) as Year_Int
        FROM csvFile
        group by Year_Int
        order by count desc
        limit 1
        """
print(ps.sqldf(q2, locals())) 

   Count  Year_Int
0    682      2015


## Question 2: What are the 3 most common offences in each year? ##

In [81]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
q="""   SELECT year, description, count 
        FROM
            (
                SELECT *, ROW_NUMBER() over (partition by year order by count desc) as row_num
                FROM(
                    SELECT year, description, count(description) as count
                    FROM csvFile
                    group by year, description
                    )
            )
WHERE row_num <=3
ORDER BY year, count desc
"""

print(ps.sqldf(q, locals()))

   year                          description  count
0  2002        FINANCIAL ID THEFT: OVER $300      1
1  2008                  FIRST DEGREE MURDER      1
2  2014  FINANCIAL IDENTITY THEFT OVER $ 300      3
3  2014                       $500 AND UNDER      2
4  2014                    CREDIT CARD FRAUD      2
5  2015                               SIMPLE     96
6  2015              DOMESTIC BATTERY SIMPLE     61
7  2015                       $500 AND UNDER     46


# EXPLANATION: #
in the previous code we had to do a 3 - level nested query.
1) the most deep level gives back the year, description columns and we created the count(description) columns that counts the description, following the path in "group by" (in group by I said: group by year and description, so the count tells me how may entries has the same dscription , for every year)
2) the second level assigns a number, increasing as the count decreases, so that I can call the last query
3) last and more external query: i selected year, description adn count from what I did before and said to the Kernel to take care only of the first 3 entries for every partition i defined in the 2nd query.

# Question 3 : What are the most common domestic offenses per year? #

In [92]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
q3="""SELECT year, description, count
        FROM
        (
        SELECT year, description, count 
        FROM
            (
                SELECT *, ROW_NUMBER() over (partition by year order by count desc) as row_num
                FROM(
                    SELECT year, description, count(description) as count, domestic
                    FROM csvFile
                    group by year, description
                    )
                    WHERE domestic = true
            )
            WHERE row_num <=3
        )  
        
            
ORDER BY year, count desc
"""

print(ps.sqldf(q3, locals()))

   year              description  count
0  2014         TELEPHONE THREAT      1
1  2015  DOMESTIC BATTERY SIMPLE     61
2  2015                OVER $500     29
3  2015  HARASSMENT BY TELEPHONE      8


## EXPLANATION ##
As the 2nd task, I needed to create a partition and order by count. This time I had to filter only on Domestical crimes, so I had to add another nested query so that i could filter using the WHERE command. Every repetition of the same command has to be done in a nested query. Can't write: where X<0, Y<0. The second statement has to be written in a nested query.

# Question 4: What are the most common offenses that lead to arrest? 

In [94]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
q4="""SELECT year, description, count
        FROM
        (
        SELECT year, description, count 
        FROM
            (
                SELECT *, ROW_NUMBER() over (partition by year order by count desc) as row_num
                FROM(
                    SELECT year, description, count(description) as count, arrest
                    FROM csvFile
                    group by year, description
                    )
                    WHERE arrest = true
            )
            WHERE row_num <=3
        )  
        
            
ORDER BY year, count desc
"""

print(ps.sqldf(q4, locals()))

   year                   description  count
0  2014                  EMBEZZLEMENT      1
1  2015  POSS: CANNABIS 30GMS OR LESS     34
2  2015                   TO PROPERTY     27
3  2015                  RETAIL THEFT     25


## Question 5: What is the most common hour of day in which offenses are recorded? ##

In [177]:
q10="""
        SELECT EXTRACT(HOUR FROM date) as hour, count(*) as count_of_offences
        FROM csvFile
        group by hour
        order by count DESC
        """

print(ps.sqldf(q10, locals()))

PandaSQLException: (sqlite3.OperationalError) near "FROM": syntax error
[SQL: 
        SELECT EXTRACT(HOUR FROM date) as hour, count(*) as count_of_offences
        FROM csvFile
        group by hour
        order by count DESC
        ]
(Background on this error at: https://sqlalche.me/e/14/e3q8)